In [1]:
import numpy as np
from helpers import *
from costs import *
from gradient_descent import *
from stochastic_gradient_descent import *


In [2]:
y, x, ids = load_csv_data(data_path="datas/train.csv", sub_sample=True)
x.shape

(5000, 30)

In [3]:
w = np.random.rand(x.shape[1])
compute_loss(y, x, w)

8255096.3165090261

In [4]:
gradient = compute_gradient(y, x, w)
gradient.shape

(30,)

In [23]:
w = np.random.rand(x.shape[1])
w, loss = least_squares(y, x, w, max_iters=10000, gamma=0.0000001)

Gradient Descent(0/9999): loss=6479839.830317005, w0=0.843728636592973, w1=0.49732034221062926
Gradient Descent(1/9999): loss=1361350.2303653355, w0=0.8121159300785203, w1=0.5036303784651381
Gradient Descent(2/9999): loss=408712.7853882518, w0=0.7900406672049812, w1=0.5059753579000098
Gradient Descent(3/9999): loss=227115.9598260727, w0=0.7722519242855079, w1=0.5066181850191881
Gradient Descent(4/9999): loss=188427.9610869171, w0=0.7564861431592869, w1=0.5065322009330597
Gradient Descent(5/9999): loss=176414.76341160072, w0=0.7417649255863538, w1=0.5061360515524895
Gradient Descent(6/9999): loss=169555.9135835394, w0=0.7276628365123493, w1=0.5056097850928716
Gradient Descent(7/9999): loss=163856.2006180753, w0=0.7139924489551348, w1=0.505030818826448
Gradient Descent(8/9999): loss=158564.02038586428, w0=0.7006688622356293, w1=0.5044324240795603
Gradient Descent(9/9999): loss=153528.86673917415, w0=0.6876514194790173, w1=0.5038288830922166
Gradient Descent(10/9999): loss=148712.32477162

Gradient Descent(104/9999): loss=18719.837183050266, w0=0.13683432966699485, w1=0.46477955843538976
Gradient Descent(105/9999): loss=18464.519147290477, w0=0.13477652126810755, w1=0.46450973361207903
Gradient Descent(106/9999): loss=18215.504054122528, w0=0.13275654861823813, w1=0.46424188781287473
Gradient Descent(107/9999): loss=17972.604468231213, w0=0.13077372281441638, w1=0.46397599878034534
Gradient Descent(108/9999): loss=17735.638791066754, w0=0.12882736787466498, w1=0.46371204451495596
Gradient Descent(109/9999): loss=17504.431074539134, w0=0.12691682047157596, w1=0.46345000327239233
Gradient Descent(110/9999): loss=17278.810840907823, w0=0.12504142967255794, w1=0.46318985356090026
Gradient Descent(111/9999): loss=17058.612908640713, w0=0.12320055668653732, w1=0.4629315741386406
Gradient Descent(112/9999): loss=16843.677224025894, w0=0.12139357461690538, w1=0.4626751440110605
Gradient Descent(113/9999): loss=16633.84869832953, w0=0.11961986822051181, w1=0.4624205424282812
Grad

Gradient Descent(214/9999): loss=7466.309266189215, w0=0.04034867163853013, w1=0.4433602517665542
Gradient Descent(215/9999): loss=7425.464547853256, w0=0.0400994226271215, w1=0.4432171617859786
Gradient Descent(216/9999): loss=7384.992677669096, w0=0.0398552011280267, w1=0.4430746781817543
Gradient Descent(217/9999): loss=7344.886801281433, w0=0.03961591425017782, w1=0.44293279500370086
Gradient Descent(218/9999): loss=7305.140249661115, w0=0.039381470771766174, w1=0.4427915063728746
Gradient Descent(219/9999): loss=7265.7465335538745, w0=0.039151781110607214, w1=0.4426508064806967
Gradient Descent(220/9999): loss=7226.699338099627, w0=0.038926757295031096, w1=0.442510689588092
Gradient Descent(221/9999): loss=7187.992517617024, w0=0.03870631293528931, w1=0.44237115002463745
Gradient Descent(222/9999): loss=7149.620090548159, w0=0.03849036319546813, w1=0.4422321821877214
Gradient Descent(223/9999): loss=7111.5762345584335, w0=0.03827882476589971, w1=0.44209378054171206
Gradient Descen

Gradient Descent(321/9999): loss=4426.060853781246, w0=0.02996674917776959, w1=0.4305983039197198
Gradient Descent(322/9999): loss=4406.005703636414, w0=0.02994950066819873, w1=0.43049690977158583
Gradient Descent(323/9999): loss=4386.059271775927, w0=0.029932878332004267, w1=0.4303957591802593
Gradient Descent(324/9999): loss=4366.220810289993, w0=0.029916869284673103, w1=0.4302948504368301
Gradient Descent(325/9999): loss=4346.489580734633, w0=0.02990146088322053, w1=0.4301941818511252
Gradient Descent(326/9999): loss=4326.864853915323, w0=0.029886640721805476, w1=0.4300937517514755
Gradient Descent(327/9999): loss=4307.345909676789, w0=0.029872396627423914, w1=0.4299935584844855
Gradient Descent(328/9999): loss=4287.93203669877, w0=0.02985871665567901, w1=0.42989360041480545
Gradient Descent(329/9999): loss=4268.622532297576, w0=0.029845589086626722, w1=0.42979387592490703
Gradient Descent(330/9999): loss=4249.416702233246, w0=0.02983300242069548, w1=0.4296943834148614
Gradient Desc

Gradient Descent(437/9999): loss=2680.5695260896596, w0=0.03008324710507331, w1=0.42013196688570864
Gradient Descent(438/9999): loss=2669.6094405579797, w0=0.030091870517351557, w1=0.4200508860681566
Gradient Descent(439/9999): loss=2658.705161426831, w0=0.030100519659910008, w1=0.41996993270632604
Gradient Descent(440/9999): loss=2647.8563964278474, w0=0.03010919326241619, w1=0.4198891062502762
Gradient Descent(441/9999): loss=2637.06285496851, w0=0.030117890081814334, w1=0.41980840615458465
Gradient Descent(442/9999): loss=2626.324248118432, w0=0.030126608901799676, w1=0.4197278318782945
Gradient Descent(443/9999): loss=2615.640288595857, w0=0.03013534853230247, w1=0.41964738288486214
Gradient Descent(444/9999): loss=2605.010690754374, w0=0.030144107808981566, w1=0.41956705864210586
Gradient Descent(445/9999): loss=2594.4351705698396, w0=0.03015288559272737, w1=0.4194868586221549
Gradient Descent(446/9999): loss=2583.9134456275083, w0=0.030161680769174002, w1=0.41940678230139933
Grad

...

Gradient Descent(9493/9999): loss=96.06769599633714, w0=0.016996413207554784, w1=0.1633962651704425
Gradient Descent(9494/9999): loss=96.05233027057454, w0=0.01699513252048933, w1=0.1633794218125757
Gradient Descent(9495/9999): loss=96.03696786590353, w0=0.016993851950591374, w1=0.1633625800355857
Gradient Descent(9496/9999): loss=96.02160878153092, w0=0.01699257149784859, w1=0.16334573983931946
Gradient Descent(9497/9999): loss=96.00625301666376, w0=0.016991291162248647, w1=0.16332890122362392
Gradient Descent(9498/9999): loss=95.99090057050924, w0=0.016990010943779218, w1=0.163312064188346
Gradient Descent(9499/9999): loss=95.97555144227478, w0=0.016988730842427975, w1=0.16329522873333271
Gradient Descent(9500/9999): loss=95.96020563116801, w0=0.016987450858182592, w1=0.16327839485843101
Gradient Descent(9501/9999): loss=95.94486313639673, w0=0.01698617099103075, w1=0.16326156256348792
Gradient Descent(9502/9999): loss=95.929523957169, w0=0.01698489124096012, w1=0.16324473184835042
G

Gradient Descent(9612/9999): loss=94.26227641370724, w0=0.016844830715163746, w1=0.1614029631240252
Gradient Descent(9613/9999): loss=94.24730038356553, w0=0.016843563884862807, w1=0.16138630681984178
Gradient Descent(9614/9999): loss=94.23232758152449, w0=0.016842297170272427, w1=0.16136965207842385
Gradient Descent(9615/9999): loss=94.2173580068145, w0=0.01684103057138045, w1=0.16135299889962007
Gradient Descent(9616/9999): loss=94.20239165866607, w0=0.01683976408817472, w1=0.16133634728327914
Gradient Descent(9617/9999): loss=94.18742853631001, w0=0.01683849772064309, w1=0.1613196972292498
Gradient Descent(9618/9999): loss=94.17246863897732, w0=0.01683723146877341, w1=0.16130304873738077
Gradient Descent(9619/9999): loss=94.15751196589913, w0=0.016835965332553528, w1=0.16128640180752082
Gradient Descent(9620/9999): loss=94.14255851630678, w0=0.0168346993119713, w1=0.16126975643951874
Gradient Descent(9621/9999): loss=94.12760828943186, w0=0.01683343340701458, w1=0.16125311263322328


Gradient Descent(9732/9999): loss=92.48798688740601, w0=0.01669363385103434, w1=0.1594153223960249
Gradient Descent(9733/9999): loss=92.47339278859292, w0=0.01669238081962939, w1=0.1593988525475193
Gradient Descent(9734/9999): loss=92.45880182690362, w0=0.01669112790248692, w1=0.15938238424372775
Gradient Descent(9735/9999): loss=92.4442140015916, w0=0.016689875099594947, w1=0.15936591748450074
Gradient Descent(9736/9999): loss=92.42962931191039, w0=0.016688622410941496, w1=0.15934945226968872
Gradient Descent(9737/9999): loss=92.41504775711377, w0=0.01668736983651459, w1=0.15933298859914222
Gradient Descent(9738/9999): loss=92.40046933645574, w0=0.016686117376302254, w1=0.15931652647271172
Gradient Descent(9739/9999): loss=92.38589404919048, w0=0.016684865030292515, w1=0.15930006589024773
Gradient Descent(9740/9999): loss=92.37132189457226, w0=0.0166836127984734, w1=0.15928360685160078
Gradient Descent(9741/9999): loss=92.35675287185568, w0=0.01668236068083294, w1=0.15926714935662145


Gradient Descent(9845/9999): loss=90.85853184497708, w0=0.016552761559752195, w1=0.15756396954713855
Gradient Descent(9846/9999): loss=90.84428756058679, w0=0.016551521364540096, w1=0.15754767330845973
Gradient Descent(9847/9999): loss=90.83004633013816, w0=0.016550281282245928, w1=0.15753137859768945
Gradient Descent(9848/9999): loss=90.81580815290553, w0=0.01654904131285788, w1=0.1575150854146798
Gradient Descent(9849/9999): loss=90.80157302816365, w0=0.016547801456364128, w1=0.15749879375928288
Gradient Descent(9850/9999): loss=90.78734095518716, w0=0.016546561712752865, w1=0.15748250363135086
Gradient Descent(9851/9999): loss=90.77311193325113, w0=0.016545322082012273, w1=0.15746621503073588
Gradient Descent(9852/9999): loss=90.75888596163064, w0=0.016544082564130546, w1=0.1574499279572901
Gradient Descent(9853/9999): loss=90.7446630396011, w0=0.01654284315909587, w1=0.15743364241086572
Gradient Descent(9854/9999): loss=90.73044316643801, w0=0.01654160386689644, w1=0.15741735839131

Gradient Descent(9958/9999): loss=89.26808434096829, w0=0.016413331287824837, w1=0.15573212864786765
Gradient Descent(9959/9999): loss=89.25418060312407, w0=0.016412103777893473, w1=0.15571600413037737
Gradient Descent(9960/9999): loss=89.24027983837945, w0=0.01641087637955369, w1=0.15569988112417527
Gradient Descent(9961/9999): loss=89.22638204602933, w0=0.016409649092793827, w1=0.1556837596291151
Gradient Descent(9962/9999): loss=89.2124872253686, w0=0.016408421917602233, w1=0.1556676396450506
Gradient Descent(9963/9999): loss=89.19859537569255, w0=0.016407194853967254, w1=0.15565152117183553
Gradient Descent(9964/9999): loss=89.1847064962965, w0=0.016405967901877233, w1=0.1556354042093237
Gradient Descent(9965/9999): loss=89.17082058647597, w0=0.01640474106132052, w1=0.15561928875736888
Gradient Descent(9966/9999): loss=89.15693764552667, w0=0.016403514332285464, w1=0.1556031748158249
Gradient Descent(9967/9999): loss=89.14305767274453, w0=0.01640228771476042, w1=0.15558706238454556

In [8]:
w, loss = least_squares_SGD(y, x, w, 10, max_iters=100, gamma=0.0000001)

Gradient Descent(1/99): loss=45258.78507090701, w0=0.11293539437844557, w1=0.6455004821126032
Gradient Descent(1/99): loss=7190.511424623448, w0=0.11118639937176304, w1=0.6451110397971282
Gradient Descent(1/99): loss=3837.0191198747643, w0=0.1094944235868044, w1=0.6449565710772031
Gradient Descent(1/99): loss=4293.6945415792725, w0=0.106969596885694, w1=0.6450228795100331
Gradient Descent(1/99): loss=7400.96843130032, w0=0.10719027662879528, w1=0.6443888756886011
Gradient Descent(1/99): loss=4043.2134814057663, w0=0.10579230988482818, w1=0.6442773453210575
Gradient Descent(1/99): loss=10266.902082366654, w0=0.10563103099283752, w1=0.6434370481621378
Gradient Descent(1/99): loss=6071.894996608618, w0=0.10341692039120715, w1=0.6433767366971894
Gradient Descent(1/99): loss=2852.8892142817126, w0=0.10229211745153118, w1=0.6435456436871679
Gradient Descent(1/99): loss=5531.618183653429, w0=0.09651865728760854, w1=0.6439977107493577
Gradient Descent(1/99): loss=2035.9673440934062, w0=0.09512

In [29]:
def get_accuracy(x, y, w):
    pred_y = predict_labels(w, x)
    correct_count = 0
    for index, yi in enumerate(y):
        pred_yi = pred_y[index]
        if pred_yi == yi:
            correct_count += 1
    return correct_count/len(y)
        
get_accuracy(x, y, w)

0.4772